<a href="https://colab.research.google.com/github/navyz/notebooks/blob/main/working_with_gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This notebook is for document purpose. No code can be run directly in here*

Shortcut on colab: Ctrl-Shift-P, Cmd-M + H

# Terms and definitions
```
SKU: Stock Keeping Unit
MTU: maximum transmission unit MTU. is the size of the largest protocol data unit (PDU) that can be communicated in a single network layer transaction
```


# gcloud configuration
```
gcloud config set project khoa-motsach-project-01
gcloud config set compute/zone us-west1-c
gcloud config set compute/region us-west1
```


# Projects


### Billing project
```
gcloud alpha billing accounts list
gcloud alpha billing projects list --billing-account=0172CB-4FCA35-4B549C
gcloud alpha billing projects unlink my-project-in-billing-account
```

### Get project information
```
gcloud projects list
gcloud projects describe khoa-motsach-project-02
gcloud projects get-ancestors-iam-policy khoa-motsach-project-02
gcloud projects get-iam-policy khoa-motsach-project-02
gcloud beta iam roles list --project=khoa-motsach-project-02
gcloud services list --enabled
gcloud compute project-info describe

gcloud compute project-info describe --project khoa-motsach-project-01
```
### Update project information
```
gcloud projects delete khoa-motsach-project-05
gcloud projects create khoa-motsach-project-05
gcloud config set project khoa-motsach-project-01
gcloud compute project-info add-metadata --metadata google-compute-default-region=asia-southeast1,google-compute-default-zone=asia-southeast1-a
```


# Virtual machine

### Basic commands

```
gcloud compute instances list
gcloud compute instances create myvm
gcloud compute instances delete myvm
gcloud compute ssh myvm
```

### Create instance
```
gcloud compute instances create instance-1 --project=khoa-motsach-project-01 --zone=asia-southeast1-a --machine-type=e2-micro --network-interface=network-tier=PREMIUM,subnet=khoa-subnet-01-03 --no-restart-on-failure --maintenance-policy=TERMINATE --preemptible --service-account=khoa-service-01@khoa-motsach-project-01.iam.gserviceaccount.com --scopes=https://www.googleapis.com/auth/cloud-platform --image=debian-10-buster-v20210817 --image-project=debian-cloud --boot-disk-size=10GB --boot-disk-type=pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any
```

### Permission
```
gcloud compute instances describe khoa-motsach-project-02
```

#### Start up script for web server
```
sudo apt update && sudo apt -y install apache2
echo '<!doctype html><html><body><h1>Hello World!</h1></body></html>' | sudo tee /var/www/html/index.html
```

### Check external IP
```
curl api.ipify.org
curl -H "Metadata-Favour:Google" metadata.google.internal/computeMetadata/v1/instance/service-accounts
```

### SSH

```
gcloud beta compute ssh --zone "asia-east1-b" "instance-3"  --project "khoa-motsach-project-01"

```




#### Add new SSH key by using different ways

1. Add a valid gcp account to project meta data
```
ssh-keygen -t rsa -f ~/.ssh/khoamotsach1 -C khoa.motsach@gmail.com
chmod 400 ~/.ssh/khoamotsach1
gcloud compute project-info describe --project khoa-motsach-project-01
create temp file with the existing ssh data, append one more line
khoa.motsach@gmail.com:ssh-rsa <publickey> khoa.motsach@gmail.com
gcloud compute project-info add-metadata --metadata-from-file ssh-keys=temp
ssh -i ~/.ssh/khoamotsach1 'khoa.motsach@gmail.com'@<external_ip>
```

2. Add a new user id to project meta data
```
ssh-keygen -t rsa -f ~/.ssh/khoamotsach2 -C khoamotsach2
chmod 400 ~/.ssh/khoamotsach2
gcloud compute project-info describe --project khoa-motsach-project-01
create temp file with the existing ssh data, append one more line
khoamotsach2:ssh-rsa <publickey> khoamotsach2
gcloud compute project-info add-metadata --metadata-from-file ssh-keys=temp
ssh -i ~/.ssh/khoamotsach2 khoamotsach2@35.240.177.47
```
  *From step 1 & 2, we learn that the user will be created as local user in VM. The permission is mapping to service account*


3. Add new user to VM meta data
```
ssh-keygen -t rsa -f ~/.ssh/khoamotsach3 -C khoamotsach3
chmod 400 ~/.ssh/khoamotsach3
 gcloud compute instances describe instance-1
create temp file with the existing ssh data, append one more line
khoamotsach3:ssh-rsa <publickey> khoamotsach3
gcloud compute instances add-metadata instance-3 --metadata-from-file ssh-keys=temp2
ssh -i ~/.ssh/khoamotsach3 khoamotsach3@35.240.177.47
```

4. Login to instance-2 from instance-1 using private ip address with new ssh key added to instance-2

  * Login to instance-1 which as firewall open, create a key-pair
```
ssh-keygen -t rsa -f ~/.ssh/khoamotsach4 -C khoamotsach4
chmod 400 ~/.ssh/khoamotsach4
```
  * In instance-1, the service account does not has permission to update meta data of instance-2, that is why we need to jump out to local to user a more powerfull user
  * On local, check what ssh key available in instace 2
```
 gcloud compute instances describe instance-2
```
  * Also on local, create temp3 file with the existing ssh data, copy all the existin ssh key and append one more line (copy the public key from the instance-1)
  * In summary, temp3 file has all the public key which allows to ssh into this instance
```
khoamotsach4:ssh-rsa <publickey> khoamotsach4
gcloud compute instances add-metadata instance-2 --metadata-from-file ssh-keys=temp3
```
  * Back to the instance-1, we can now ssh into instance-2, base on the ssh key we just added. Both public/private IP should work
```
ssh -i ~/.ssh/khoamotsach4 khoamotsach4@35.247.173.15
ssh -i ~/.ssh/khoamotsach4 khoamotsach4@192.168.3.3
```
5. Login to instance-3 from instance-1 using private ip address, no external ip available
  * From the previous lab, instance-2 has both internal and external IP Address. This lab is to test if we can ssh into a instance which only has private IP.
  * When create instance-3, remember not to create public IP address
  * Repeat the same steps as above

  * Login to instance-1 which as firewall open, create a key-pair
```
ssh-keygen -t rsa -f ~/.ssh/khoamotsach5 -C khoamotsach5
chmod 400 ~/.ssh/khoamotsach5
```
  * On local, add the ssh key into instance-3
```
gcloud compute instances describe instance-3
```
  * Create temp4 file with the existing ssh data, append one more line (copy the public key from the instance-1)
```
khoamotsach5:ssh-rsa <publickey> khoamotsach5
gcloud compute instances add-metadata instance-3 --metadata-from-file ssh-keys=temp4
```
  * Back to the instance-1, ssh to instance-3
```
ssh -i ~/.ssh/khoamotsach5 khoamotsach5@192.168.3.4
```

# IAM

### Roles
```
gcloud beta iam roles list
```

### Service accounts
```
gcloud iam service-accounts list
gcloud iam service-accounts delete khoa-service-02@khoa-motsach-project-01.iam.gserviceaccount.com
```



# VPC
```
gcloud compute networks list
gcloud compute networks delete khoa-vpc-01
gcloud compute firewall-rules list
gcloud compute firewall-rules delete allow-incomming
gcloud compute networks subnets list
gcloud compute networks subnets delete khoa-subnet-01 --region=asia-east1
```

### Networks

#### Create VPC
```
gcloud compute networks create khoa-vpc-01 --project=khoa-motsach-project-01 --subnet-mode=custom --mtu=1460 --bgp-routing-mode=regional
```


#### Subnet
```
gcloud compute networks subnets create khoa-subnet-01-01 --project=khoa-motsach-project-01 --range=192.168.1.0/24 --network=khoa-vpc-01 --region=asia-east1 --enable-private-ip-google-access

gcloud compute networks subnets list
```

#### Create subnet without private ip access
```
gcloud compute networks subnets create khoa-subnet-01-02 --project=khoa-motsach-project-01 --range=192.168.2.0/24 --network=khoa-vpc-01 --region=asia-east1
```

### Firewall
```
gcloud compute firewall-rules list

gcloud compute --project=khoa-motsach-project-01 firewall-rules create web-firewall --direction=INGRESS --priority=1000 --network=khoa-vpc-01 --action=ALLOW --rules=tcp:80 --source-ranges=0.0.0.0/0 --target-service-accounts=khoa-service-01@khoa-motsach-project-01.iam.gserviceaccount.com

gcloud compute --project=khoa-motsach-project-01 firewall-rules create ssh-rule --direction=INGRESS --priority=1000 --network=khoa-vpc-01 --action=ALLOW --rules=tcp:22 --source-ranges=0.0.0.0/0 --target-service-accounts=khoa-service-01@khoa-motsach-project-01.iam.gserviceaccount.com

gcloud compute --project=khoa-motsach-project-01 firewall-rules create ssh-private --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:22 --source-ranges=192.168.1.0/24,192.168.2.0/24
```

### Cloud NAT
```
gcloud compute routers create nat-router --network khoa-vpc-01 --region asia-southeast1
gcloud compute routers nats create nat-config --router-region asia-southeast1 --router nat-router --nat-all-subnet-ip-ranges --auto-allocate-nat-external-ips
```

### VM without external IP
- ssh from local: Yes. Need firewall open to internet & using gcloud compute ssh
- download from internet: need cloud NAT (previous steps)
- ssh from same subnet: need firewall open to intranet & ssh key register & using tranditional ssh
  -  require IP forwarding: NO. IP forwarding is just be used for the NAT gateway VM
- Need  project-wide SSH keys? NO, VM ssh key will do
- Need  VM SSH keys? NO, project-wide ssh  key will do